# Geoinformática - Práctica 6
## Agrupamiento, Regionalización y Segmentación Geodemográfica

## Parte 2 - Homogeneidad y Correlación de Variables

En la sección anterior de la práctica, se estudiaron los conceptos generales relacionados con el Agrupamiento de datos, así como de su variante espacial en la Regionalización. En esta parte, se analizarán con un poco más de detalle alguna de las particularidades de los procesos estudiados; en específico, se estudiarán algunas formas de discernir las variables correctas para realizar una Regionalización, así como algunos parámetros para conocer si los resultados obtenidos son lo suficientemente aceptables para ser considerados como válidos o ciertos.

In [1]:
# Librerías a Utilizar
import seaborn as sns
import pandas as pd
import pysal as ps
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cluster

/home/datalab/miniconda3/envs/geoinf/lib/python3.7/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


___
## Generalización contra Detalle
Cuando se ejecuta algún algoritmo de Regionalización, en el transfondo se está realizando una generalización sobre el espacio, esto es, se está asignando un mismo *valor* o atributo a áreas que resultan más grandes que las que se tenían en un inicio.

Esta Generalización puede resultar positiva, pues permite visualizar algunos patrones reproducibles en el espacio y, por ende, permitir deducir si la componente espacial es significativa sobre el fenómeno estudiado, dando otra herramienta para explicarlo. Por otra parte, el generalizar provoca que se pierda el detalle que se tenía inicialmente con áreas más pequeñas y, por ende, la explicación que pueda derivarse también pierde de profundidad. Como tal, es importante encontrar un balance entre el poder explicativo que se gana al momento de Regionalizar y la pérdida de profundidad implícita en la Generalización.

Para estudiar este fenómeno, se tomarán nuevamente los datos utilizados en la sección anterior de la práctica, esto es, las calificaciones de los AirBnb contenidos en las AGEB's de la Alcaldía Cuauhtémoc de la CDMX. Como tal, es necesario importar estos datos:

In [2]:
airbnb = gpd.read_file('data/agebs_airbnb.shp')  # Importar el ShapeFile
airbnb = airbnb.set_index('ageb')                # Establecer la Clave Geográfica del AGEB como Índice
airbnb.head()                                    # Observar el GeoDataFrame importado

,no_prop,calif,expec,limp,checkin,com,ubi,precio,geometry
ageb,,,,,,,,,
0901500010235,4.0,96.5000,10.00000,10.00000,10.00000,10.00000,10.00000,9.75000,"POLYGON ((484786.4564242049 2151318.632194062,..."
0901500010362,1.0,100.0000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,"POLYGON ((484415.3002532361 2150990.921335925,..."
0901500010606,14.0,96.3571,9.92857,10.00000,10.00000,9.92857,9.71429,9.85714,"POLYGON ((483366.716567839 2150099.090658844, ..."
0901500011017,204.0,96.1569,9.83823,9.64706,9.86765,9.86765,9.92157,9.68627,"POLYGON ((483869.0666524998 2147955.234262425,..."
0901500010201,1.0,100.0000,9.00000,10.00000,10.00000,10.00000,10.00000,10.00000,"POLYGON ((484252.7970705956 2151134.263629142,..."


En la siguiente celda se repetirán los pasos ejecutados en la sección anterior de la práctica para generar una Regionalización; en este caso, se generarán 15 Regiones utilizando el Algoritmo *AZP*. Asimismo, se utilizará sólo una de las variables anteriores, la de Calificación General (`calif`), para facilitar el análisis posterior:

In [4]:
# Importar la librería local 'clusterpy'
import clusterpy

# Generar el objeto de tipo 'layer' que la librería necesita
layer = clusterpy.importArcData('data/agebs_airbnb')

# Ejecutar el Algoritmo AZP de Regionalización
layer.cluster('azp', ['calif'], 15, wType='queen')

# Indicar a qué región pertenece cada AGEB como una nueva columna
airbnb['azpcls_15'] = layer.region2areas

# Visualizar el resultado
airbnb.head()

Loading data/agebs_airbnb.dbf
Loading data/agebs_airbnb.shp
Done
Getting variables
Variables successfully extracted
Running original AZP algorithm (Openshaw and Rao, 1995)
Number of areas:  153
Number of regions:  15
Constructing regions
initial Solution:  [2, 2, 2, 3, 2, 7, 2, 7, 2, 7, 2, 4, 7, 7, 2, 7, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 7, 7, 2, 2, 2, 2, 2, 2, 3, 6, 11, 3, 3, 6, 2, 3, 2, 2, 2, 2, 7, 2, 14, 13, 2, 2, 7, 2, 4, 9, 9, 2, 2, 2, 2, 2, 2, 5, 2, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 3, 10, 3, 3, 10, 3, 3, 3, 3, 3, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 12, 0, 3, 6, 3, 3, 3, 3, 3, 11, 11, 3, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 3, 3, 3, 3, 3, 3, 4, 2, 2, 2, 7, 2, 2, 2, 2, 1, 1, 14]
initial O.F:  844.8615366219416
Performing local search
FINAL SOLUTION:  [2, 2, 2, 3, 2, 7, 2, 7, 2, 7, 2, 4, 7, 7, 2, 7, 2, 2, 2, 2, 0, 3, 0, 0, 3, 3, 0, 2, 7, 7, 2, 2, 2, 2, 2, 2, 3, 6, 11, 3, 0, 6, 2, 3, 2, 2, 2, 2, 7, 2, 14, 13, 2, 2, 7, 2, 4, 7, 9, 2, 2, 2, 2, 2, 2, 5,

,no_prop,calif,expec,limp,checkin,com,ubi,precio,geometry,azpcls_15
ageb,,,,,,,,,,
0901500010235,4.0,96.5000,10.00000,10.00000,10.00000,10.00000,10.00000,9.75000,"POLYGON ((484786.4564242049 2151318.632194062,...",2
0901500010362,1.0,100.0000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,"POLYGON ((484415.3002532361 2150990.921335925,...",2
0901500010606,14.0,96.3571,9.92857,10.00000,10.00000,9.92857,9.71429,9.85714,"POLYGON ((483366.716567839 2150099.090658844, ...",2
0901500011017,204.0,96.1569,9.83823,9.64706,9.86765,9.86765,9.92157,9.68627,"POLYGON ((483869.0666524998 2147955.234262425,...",3
0901500010201,1.0,100.0000,9.00000,10.00000,10.00000,10.00000,10.00000,10.00000,"POLYGON ((484252.7970705956 2151134.263629142,...",2


Como se estudió en la práctica anterior, es posible obtener Estadísticos Descriptivos tanto de la variable original, utilizada para la Regionalización, como de las regiones estudiadas. En el caso de la variable original, sus estadísticos pueden obtenerse a través de la función `.describe()` estudiada anteriormente:

In [13]:
airbnb['calif'].describe()

count    153.000000
mean      94.668812
std        4.188733
min       76.750000
25%       93.714300
50%       95.350700
75%       96.589700
max      100.000000
Name: calif, dtype: float64

Para el análisis, únicamente resulta de interés obtener la *Desviación Estándar* ($\sigma^2$ o `std`) de la variable. Para esto, únicamente es necesario llamar la función `.std()`, que permite aislar a este estadístico en específico:

In [15]:
airbnb['calif'].std()

4.188732594789283

Lo que el estadístico anterior pretende comunicar es el rango de valores en el cual se encuentran la mayoría de los datos, tomando como referencia la media de los mismos. En el ejemplo de trabajo, teniendo una Desviación Estándar de 4.18 y una Media de 94.67 para la variable, puede asegurarse que la mayoría de las Calificaciones Generales de los AirBnb se encuentran en un rango de $94.67 \pm 4.18$, en otras palabras, entre 90.49 y 98.85. Lo anterior ayuda a tener una idea de qué tan dispersos se encuentran los datos; mientras más dispersos, mayor será la Desviación Estándar y, por ende, menos similares son los datos entre sí.

Para el caso de las regiones, es necesario utilizar primero la función `.groupby()`, agrupando a través de la etiqueta, para después poder obtener los estadísticos descriptivos de interés; cabe notar el uso del doble corchete para aislar únicamente a la variable utilizada en la regionalización y la de referencia para el agrupamiento:

In [16]:
airbnb[['calif' , 'azpcls_15']].groupby('azpcls_15').describe()

calif                                                         \
          count        mean       std       min         25%        50%   
azpcls_15                                                                
0          29.0   94.834372  0.732723   92.9608   94.529400   94.95890   
1           2.0   80.500000  2.121320   79.0000   79.750000   80.50000   
2          47.0   95.298849  3.321830   85.0000   93.770850   95.39890   
3          40.0   96.605582  1.478398   94.0000   95.842525   96.18655   
4           3.0   85.366667  2.150194   83.2000   84.300000   85.40000   
5           1.0  100.000000       NaN  100.0000  100.000000  100.00000   
6           6.0   89.304733  3.677714   82.0000   90.000000   90.15000   
7          12.0   98.208333  2.332657   93.0000   96.937500   99.37500   
8           2.0   79.541650  3.947989   76.7500   78.145825   79.54165   
9           1.0   95.750000       NaN   95.7500   95.750000   95.75000   
10          2.0   93.128850  0.769686   92.5846   92.856725   93.12885   
11          3.0   91.555533  1.548584   90.5000   90.666650   90.83330   
12          2.0   88.821450  2.929473   86.7500   87.785725   88.82145   
13          1.0   93.000000       NaN   93.0000   93.000000   93.00000   
14          2.0   86.516650  3.087723   84.3333   85.424975   86.51665   

                                 
                  75%       max  
azpcls_15                        
0           95.186400   96.4375  
1           81.250000   82.0000  
2           97.117650  100.0000  
3           97.031250  100.0000  
4           86.450000   87.5000  
5          100.000000  100.0000  
6           91.153125   92.0909  
7          100.000000  100.0000  
8           80.937475   82.3333  
9           95.750000   95.7500  
10          93.400975   93.6731  
11          92.083300   93.3333  
12          89.857175   90.8929  
13          93.000000   93.0000  
14          87.608325   88.7000

Nuevamente, si se desea obtener únicamente la Desviación Estándar, basta con llamar sólo a `.std()`:

In [19]:
airbnb[['calif' , 'azpcls_15']].groupby('azpcls_15').std()

,calif
azpcls_15,
0,0.732723
1,2.121320
2,3.321830
3,1.478398
4,2.150194
5,NaN
6,3.677714
7,2.332657
8,3.947989


En primer lugar, es inmediatamente visible la influencia de la Regionalización sobre la variable de interés. En la gran mayoría de los casos, la Desviación Estándar de las regiones es menor a la Desviación Estándar de la variable original (4.18); esto significa que, al momento de Regionalizar, el algoritmo agrupó AGEB's con Calificaciones Generales muy similares, de modo que la variabilidad de los datos disminuyera.

Lo tabla anterior también permite dar una primera idea de cuál es la variación general dentro de cada una de las regiones. De forma muy superficial, podría decirse que las regiones con una menor Desviación Estándar son más Homogéneas, es decir, los AirBnb en su interior se asemejan más entre sí; por el contrario, las regiones con una Desviación Estándar mayor poseen AirBnb no tan similares.